In [3]:
import requests
import json
import re
import networkx as nx
%matplotlib inline
import pandas as pd

In [4]:
# API key = api-key=A9IKQgYHxfUzG2FCHgpzO4kBypcsV67w# 

def req2():
    requestUrl = "https://api.nytimes.com/svc/archive/v1/2020/11.json?api-key=A9IKQgYHxfUzG2FCHgpzO4kBypcsV67w"
    #requestHeaders = {"Accept": "application/json"} headers=requestHeaders
    #query = {"year":2020, "month":2}
    
    response = requests.get(requestUrl)

    #print(response.text)
    response2 = response.json()
    return response2
    

response = req2()

In [5]:
z = response['response']['docs']

In [46]:
o = 'By Keith Collins, Trip Gabriel and Stephanie Saul'
g = 'By Nick Corasaniti'

In [6]:
def splitstring(x):
    
    #o = 'By Keith Collins, Trip Gabriel and Stephanie Saul'
    h = x.replace('By ', '')
    s = h.replace(', ', ',')
    t = s.replace(' and ', ',').split(',')
    print(t)


In [48]:
splitstring(o)
splitstring(g)

['Keith Collins', 'Trip Gabriel', 'Stephanie Saul']
['Nick Corasaniti']


In [7]:
author_name_map = {}
author_genre_map = {}

In [8]:
#def att(x):
for x in z:
    type_of_material = x['type_of_material']
    j1 = x['byline']['original']
    h = j1.replace('By ', '')
    s = h.replace(', ', ',')
    t = s.replace(' and ', ',').split(',')
     
    # For each author, add the genre to that author's list
    for r in t:
        if r != '':
            author_name_map[r] = r
        
        
            this_authors_genres = author_genre_map.get(r, {})
        #print(this_authors_genres)
            
            # Increment the count of genres for this actor
        #for g in type_of_material:
            if type_of_material != '':
                this_authors_genres[type_of_material] = this_authors_genres.get(type_of_material, 0) + 1
        #print(this_authors_genres)
        
         # Update the map
                author_genre_map[r] = this_authors_genres


In [138]:
#for k, v in author_genre_map.items():
    #print(f'key:{k}, Value:{v}')
    #print(k, v)

In [9]:
author_genre_map['Kyle Turner']

{'Review': 2}

In [182]:
# Get all actors as an index for a dataframe
index = author_genre_map.keys()

# Get the genre-counts for each actor in the index
rows = [author_genre_map[k] for k in index]

# Create the data frame from these rows, with the actors as index
df = pd.DataFrame(rows, index=index)

# Fill NAs with zero, as NA means the actor has not starred in that genre
df = df.fillna(0)

df

,News,Interactive Feature,Review,News Analysis,Video,An Appraisal,briefing,Editorial,Obituary (Obit),Op-Ed,List,Letter,Biography
Nick Corasaniti,38.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Keith Collins,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Trip Gabriel,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Stephanie Saul,20.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Jennifer Medina,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Deb Willis,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Alex Harrington,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Richard Frishman,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Hugh Eakin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


### Apply k-Means with a Fixed K

In [11]:
from sklearn.cluster import KMeans

In [161]:
k = 7

In [162]:
cluster_model = KMeans(n_clusters=k)

In [163]:
cluster_model.fit(df)

KMeans(n_clusters=7)

In [164]:
cluster_labels = cluster_model.predict(df)
authors_cluster_df = pd.DataFrame(cluster_labels, index=df.index, columns=["cluster frequency"])

In [183]:
# create a table. two column (cluster ID and frequency) and rows cluster 
gg = pd.DataFrame(authors_cluster_df["cluster frequency"].value_counts())
gg

,cluster frequency
0,966
6,272
1,126
3,32
5,9
2,8
4,2


In [185]:
for cluster, authors in authors_cluster_df.groupby("cluster frequency"):
    print("Cluster:", cluster, "Size:", authors.shape[0])
    
    for a_id in authors.sample(5).index:
        print("\t", author_name_map[a_id])

Cluster: 0 Size: 966
	 Chris Azzopardi
	 Melena Ryzik
	 Ivan Nechepurenko
	 Ian Bostridge
	 Lori Soderlind
Cluster: 1 Size: 126
	 Benedict Carey
	 Michael Gold
	 Maggie Astor
	 M.D
	 Mike Isaac
Cluster: 2 Size: 8
	 Nick Corasaniti
	 Katie Glueck
	 Annie Karni
	 Maggie Haberman
	 Michael D. Shear
Cluster: 3 Size: 32
	 Jonathan Martin
	 Mark Landler
	 David E. Sanger
	 Deb Amlen
	 Isabella Kwai
Cluster: 4 Size: 2


ValueError: Cannot take a larger sample than population when 'replace=False'

In [186]:
# cluster 0
index_labels = ['Chris Azzopardi', 'Melena Ryzik', 'Ivan Nechepurenko', 'Ian Bostridge', 'Lori Soderlind']
ff = pd.DataFrame(df,index=index_labels)
ff

,News,Interactive Feature,Review,News Analysis,Video,An Appraisal,briefing,Editorial,Obituary (Obit),Op-Ed,List,Letter,Biography
Chris Azzopardi,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Melena Ryzik,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ivan Nechepurenko,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ian Bostridge,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Lori Soderlind,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [187]:
# cluster 1
# cluster 0
index_labels1 = ['Benedict Carey', 'Michael Gold', 'Maggie Astor', 'M.D', 'Mike Isaac']

ff1 = pd.DataFrame(df, index=index_labels1)
ff1

,News,Interactive Feature,Review,News Analysis,Video,An Appraisal,briefing,Editorial,Obituary (Obit),Op-Ed,List,Letter,Biography
Benedict Carey,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Michael Gold,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Maggie Astor,15.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
M.D,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mike Isaac,13.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [180]:
# cluster 1
# cluster 0
index_labels2 = ['Kwame Opam', 'Billy Witz']

ff2 = pd.DataFrame(df, index=index_labels2)
ff2.tail(-1)

,News,Interactive Feature,Review,News Analysis,Video,An Appraisal,briefing,Editorial,Obituary (Obit),Op-Ed,List,Letter,Biography
Billy Witz,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
